In [1]:
import xlrd
import os 
import sqlite3
import csv

In [2]:
db_path = './datawarehouse.db'
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT * from weather'''

cur.execute(query)
r = cur.fetchone()

print(r)

con.close()

('2022-01-01', 17.5, 12.3, 14.6, 17.5, 12.3, 14.6, 13.3, 92.1, 11.399, 100, 37.5, 'rain', 0, 0, '', 18.4, 190.1, 1007.2, 99.5, 10, 21.4, 0.6, 0, '', '2022-01-01T07:26:53', '2022-01-01T16:56:53', 0.96, 'Rain, Overcast', 'Cloudy skies throughout the day with rain.', 'rain', 'KDCA,72405013743,72403793728,F0198,KADW,KDAA,AS365,74594013705')


# Backup

In [10]:
# Full path to xls, including filename
xls_path = './primary/airbnb_data_dict.xls'
airbnb_file = xlrd.open_workbook_xls(xls_path)
airbnb_var_sheets = airbnb_file.sheets()

variable_dict = {}
print('Saved Data Dictonaries:')
for s in airbnb_var_sheets:
    s_name = s.name
    var_stored = True
    
    if s_name.rfind('listings.csv detail v4') != -1:
        variable_dict['listings'] = s
        
    elif s_name.rfind('reviews.csv v1') != -1:
        variable_dict['reviews'] = s
        
    elif s_name.rfind('calendar.csv v2') != -1:
        variable_dict['calendar'] = s
        
    else:
        var_stored = False
        
    if var_stored:
        print(f'\t{s_name}')

Saved Data Dictonaries:
	listings.csv detail v4.3
	listings.csv detail v4
	reviews.csv v1
	calendar.csv v2


In [49]:
# Automatically create the listings table
cur_sheet = variable_dict['listings']
field_vals = cur_sheet.col_values(0)[8:82]
type_vals = cur_sheet.col_values(1)[8:82]

out = f'''-- Table: listings
DROP TABLE IF EXISTS listings;
CREATE TABLE listings(
    id int PRIMARY KEY,\n'''

for val in range (1, len(field_vals)-1):
    var_type = type_vals[val]
    if var_type == 'boolean [t=true; f=false]':
        var_type = 'boolean'
    if var_type == '':
        var_type = 'blob'
    out+=f'''    {field_vals[val]} {var_type},\n'''
    
out+=f'''    {field_vals[len(field_vals)-1]} {var_type}
);\n\n'''

In [50]:
print(len(field_vals))

74


In [45]:
# Automatically create the reviews table
cur_sheet = variable_dict['reviews']
field_vals = cur_sheet.col_values(0)[8:14]
type_vals = cur_sheet.col_values(1)[8:14]

out += f'''-- Table: reviews
DROP TABLE IF EXISTS reviews;
CREATE TABLE reviews(
    id int PRIMARY KEY,
    listing_id int,\n'''

for val in range (1, len(field_vals)-1):
    var_type = type_vals[val]
    if var_type == 'boolean [t=true; f=false]':
        var_type = 'boolean'
    if var_type == '':
        var_type = 'blob'
    out+=f'''    {field_vals[val]} {var_type},\n'''
    
out+=f'''    {field_vals[len(field_vals)-1]} {var_type}
);\n\n'''

In [46]:
# Automatically create the reviews table
cur_sheet = variable_dict['calendar']
field_vals = cur_sheet.col_values(0)[8:15]
type_vals = cur_sheet.col_values(1)[8:15]

out += f'''-- Table: calendar
DROP TABLE IF EXISTS calendar;
CREATE TABLE calendar(
    id int PRIMARY KEY,
    listing_id int,\n'''

for val in range (1, len(field_vals)-1):
    var_type = type_vals[val]
    if var_type == 'boolean [t=true; f=false]':
        var_type = 'boolean'
    if var_type == '':
        var_type = 'blob'
    if field_vals[val]!='id':
        out+=f'''    {field_vals[val]} {var_type},\n'''
    
out+=f'''    {field_vals[len(field_vals)-1]} {var_type}
);\n\n'''

In [47]:
text_file = open("datawarehouse.sql", "w")
n = text_file.write(out)
text_file.close()

In [38]:
field_vals

['listing_id',
 'date',
 'available',
 'price',
 'adjusted_price',
 'minimum_nights',
 'maximum_nights']

In [ ]:


out = f'''-- Table: listings
DROP TABLE IF EXISTS listings;
CREATE TABLE listings(
    id int PRIMARY KEY,\n'''

